# <b>Avalia o comportamento das Requests</b>

In [1]:
# Importando as dependências
from quantumnet.components import Controller, Network
from quantumnet.objects import time
from simulations import *
logger.deactivate()

Informações a serem coletadas

In [2]:
received_requests = {}
successful_requests = {}
fulfilled_requests = {}

### <b>Preenchimento Proativo</b>

In [3]:
# Parâmetros das simulações
network_info = {
    'n_initial_qubits': 10,
    'n_initial_eprs': 10,
    'topology_name': 'Grade',
    'topology_params': (3, 4),
    'time_to_refill': 10,
}
controller_info = {
    'default_ttl': 50,
}
request_info = {
    'num_hosts': 12,
    'n_requests': 100,
    'traffic_type': 'random',
    'burst_probability': 0.3,
    'burst_size': 25,
    'requests_per_burst': 5,
    'fmin_range': (0.5, 1.0),
    'eprs_range': (1, 10),
}
# Exclusivo para preenchimento proativo e hibrido
proactive_params = {
    1 : { 'frange': (0.6, 0.9), 'neprs': 10 },
}

In [ ]:
# Inicializando a simulação
sim = Sim(network_info, controller_info, request_info)
sim.requests = generate_traffic(request_info)

print(f"Requests: {sim.requests}")

# Preenche as tabelas de forma proativa
for alice in sim.network.hosts:
    print(f"Adicionando regras para {alice}")
    for bob in sim.network.hosts:
        for key in proactive_params.keys():
            sim.controller.add_match_route_rule_in_host_proactive(alice, bob, proactive_params[key]['frange'], proactive_params[key]['neprs'])
    # Mostra as tabelas
    sim.network.get_host(alice).draw_flow_table()

Requests: [RE8A600:4-0:0.59:2, RE8A840:7-1:0.87:4, RE8A870:0-11:0.71:2, RE8A8A0:0-7:0.76:4, RE8A8D0:0-7:0.6:3, RE8A900:5-2:0.53:1, RE8A930:3-7:0.53:8, RE8A960:8-11:0.59:6, RE8A990:8-2:0.89:9, RE8A9C0:8-6:0.79:6, RE8A9F0:3-1:0.59:10, RE8AA20:8-5:0.73:1, RE8AA50:3-4:0.78:2, RE8AA80:10-0:0.61:9, RE8AAB0:4-8:0.57:9, RE8AAE0:11-4:0.75:9, RE8AB10:7-11:0.86:10, RE8AB40:0-1:0.54:3, RE8AB70:7-1:0.78:5, RE8ABA0:5-6:0.69:10, RE8ABD0:0-3:0.9:2, RE8AC00:0-7:0.69:6, RE8AC30:9-0:0.83:3, RE8AC60:6-3:0.83:7, RE8AC90:6-9:0.94:2, RE8ACC0:6-3:0.81:2, RE8ACF0:11-7:0.99:9, RE8AD20:9-2:0.93:7, RE8AD50:4-11:0.96:1, RE8AD80:7-6:0.54:1, RE8ADB0:3-11:0.83:6, RE8ADE0:11-6:0.59:8, RE8AE10:7-11:0.72:6, RE8AE40:8-5:0.99:6, RE8AE70:11-3:0.88:10, RE8AEA0:10-3:0.58:9, RE8AED0:5-9:0.95:10, RE8AF00:1-6:0.63:10, RE8AF30:9-6:0.88:7, RE8AF60:11-9:0.72:8, RE8AF90:10-8:0.89:5, RE8AFC0:1-6:0.84:6, RE8AFF0:0-2:0.77:10, RE8B020:3-11:0.98:4, RE8B050:6-9:0.53:2, RE8B080:9-2:0.66:7, RE8B0B0:6-3:0.63:9, RE8B0E0:8-11:0.82:8, RE8B110:

In [5]:
# Processa as requisições
for request in sim.requests:
    print(f"[Time {time.get_current_time()}] Processando requisição {request}.")
    alice = sim.network.get_host(request.alice)
    rule = alice.find_rule_by_request(request)

    if rule == False:  # Caso não exista um match na tabela
        print(f"[Time {time.get_current_time()}] Descartando requisição {request} no Host {alice}.")
        request.starttime = time.get_current_time()
        request.endtime = time.get_current_time()
        # Registra a requisição descartada
        sim.controller.fulfill_request(request)
        
    else:  # Caso já exista a regra
        print(f"[Time {time.get_current_time()}] Atendendo requisição {request} no Host {alice}.")
        request.starttime = time.get_current_time()
        sim.controller.run_rule(rule[1])
        request.endtime = time.get_current_time() + 1        
        # Registra a requisição atendida
        sim.controller.successful_request(request)
        # Exibir informações da requisição
        print(f"Request {request}: Start Time = {request.starttime}, End Time = {request.endtime}")
        
    # Atualiza o tempo
    sim.update_time(1)

# Salva as requisições
received_requests['proativo'] = sim.requests.copy()
successful_requests['proativo'] = sim.controller.successful_requests
fulfilled_requests['proativo'] = sim.controller.fulfilled_requests

# Fim da simulação
time.reset()


[Time 0] Processando requisição RE8A600:4-0:0.59:2.
[Time 0] Descartando requisição RE8A600:4-0:0.59:2 no Host 4.


AttributeError: 'Sim' object has no attribute 'filename'

### <b>Preenchimento Reativo</b>

In [6]:
# Parâmetros das simulações
network_info = {
    'n_initial_qubits': 10,
    'n_initial_eprs': 10,
    'topology_name': 'Grade',
    'topology_params': (3, 4),
    'time_to_refill': 10,
}
controller_info = {
    'default_ttl': 50,
}
request_info = {
    'num_hosts': 12,
    'n_requests': 100,
    'traffic_type': 'random',
    'burst_probability': 0.3,
    'burst_size': 25,
    'requests_per_burst': 5,
    'fmin_range': (0.5, 1.0),
    'eprs_range': (1, 10),
}

In [ ]:
# Inicializando a simulação
sim = Sim(network_info, controller_info, request_info)
print("Requests:", sim.requests)

for request in sim.requests:
        print(f"[Time {time.get_current_time()}] Processando requisição {request}...")
        alice = sim.network.get_host(request.alice)
        rule = alice.find_rule_by_request(request)

        if rule == False:  # Caso não exista um match na tabela
            request.starttime = time.get_current_time()
            sim.update_time(3)
            alice.draw_flow_table()
            print(f"[Time {time.get_current_time()}] Adicionando regra no Host {alice}")
            sim.controller.add_match_route_rule_in_host_reactive(request)
            alice.draw_flow_table()
            rule = alice.find_rule_by_request(request)
            
        else:  # Caso já exista a regra
            print(f"[Time {time.get_current_time()}] Regra existente para {request} no Host {alice}.")
            request.starttime = time.get_current_time()
            sim.update_time(1)
                
        # Executa a regra
        print(f"[Time {time.get_current_time()}] Atendendo requisição {request} no Host {alice}.")
        sim.controller.run_rule(rule[1])
        request.endtime = time.get_current_time()
        # Registra a requisição atendida
        sim.controller.successful_request(request)
        
        # Exibir informações da requisição
        print(f"[Time {time.get_current_time()}] Request {request}: Start Time = {request.starttime}, End Time = {request.endtime}")

# Salva as requisições
received_requests['reativo'] = sim.requests.copy()
successful_requests['reativo'] = sim.controller.successful_requests
fulfilled_requests['reativo'] = sim.controller.fulfilled_requests

# Fim da simulação
time.reset()

### <b>Preenchimento Híbrido</b>

In [8]:
# Parâmetros das simulações
network_info = {
    'n_initial_qubits': 10,
    'n_initial_eprs': 10,
    'topology_name': 'Grade',
    'topology_params': (3, 4),
    'time_to_refill': 10,
}
controller_info = {
    'default_ttl': 50,
}
request_info = {
    'num_hosts': 12,
    'n_requests': 100,
    'traffic_type': 'random',
    'burst_probability': 0.3,
    'burst_size': 25,
    'requests_per_burst': 5,
    'fmin_range': (0.5, 1.0),
    'eprs_range': (1, 10),
}
# Exclusivo para preenchimento proativo e hibrido
proactive_params = {
    1 : { 'frange': (0.6, 0.7), 'neprs': 10 },
    5:  { 'frange': (0.8, 0.9), 'neprs': 10 },
}

In [ ]:
# Inicializando a simulação
sim = Sim(network_info, controller_info, request_info)

# Preenche as tabelas de forma proativa
for alice in sim.network.hosts:
    print(f"Adicionando regras para {alice}")
    for bob in sim.network.hosts:
        for key in proactive_params.keys():
            sim.controller.add_match_route_rule_in_host_proactive(alice, bob, proactive_params[key]['frange'], proactive_params[key]['neprs'])
    # Mostra as tabelas
    sim.network.get_host(alice).draw_flow_table()

In [ ]:
# Inicializando a simulação
sim = Sim(network_info, controller_info, request_info)
print("Requests:", sim.requests)

for request in sim.requests:
        print(f"[Time {time.get_current_time()}] Processando requisição {request}...")
        alice = sim.network.get_host(request.alice)
        rule = alice.find_rule_by_request(request)

        if rule == False:  # Caso não exista um match na tabela
            request.starttime = time.get_current_time()
            sim.update_time(3)
            alice.draw_flow_table()
            print(f"[Time {time.get_current_time()}] Adicionando regra no Host {alice}")
            sim.controller.add_match_route_rule_in_host_reactive(request)
            alice.draw_flow_table()
            rule = alice.find_rule_by_request(request)
            
        else:  # Caso já exista a regra
            print(f"[Time {time.get_current_time()}] Regra existente para {request} no Host {alice}.")
            request.starttime = time.get_current_time()
            sim.update_time(1)
                
        # Executa a regra
        print(f"[Time {time.get_current_time()}] Atendendo requisição {request} no Host {alice}.")
        sim.controller.run_rule(rule[1])
        request.endtime = time.get_current_time()
        # Registra a requisição atendida
        sim.controller.successful_request(request)
        
        # Exibir informações da requisição
        print(f"[Time {time.get_current_time()}] Request {request}: Start Time = {request.starttime}, End Time = {request.endtime}")

# Salva as requisições
received_requests['hibrido'] = sim.requests.copy()
successful_requests['hibrido'] = sim.controller.successful_requests
fulfilled_requests['hibrido'] = sim.controller.fulfilled_requests

# Fim da simulação
time.reset()

In [ ]:
import matplotlib.pyplot as plt

def calcular_percentual_atendimento(requisicoes_recebidas, requisicoes_atendidas):
    # Cálculo do percentual de atendimento por modo
    percentual_atendido = {}
    for modo in requisicoes_recebidas:
        total_recebidas = len(requisicoes_recebidas[modo])
        total_atendidas = len(requisicoes_atendidas[modo])
        percentual_atendido[modo] = (total_atendidas / total_recebidas) * 100 if total_recebidas > 0 else 0
    return percentual_atendido

percentual = calcular_percentual_atendimento(received_requests, successful_requests)

# Plotagem do gráfico
plt.figure(figsize=(8, 6))
plt.bar(percentual.keys(), percentual.values(), color=['#4c72b0', '#55a868', '#c44e52'])

plt.xlabel('Modo de Preenchimento')
plt.ylabel('Percentual de Requisições Atendidas (%)')
plt.ylim(0, 100)

# Exibe o valor de cada barra
for i, v in enumerate(percentual.values()):
    plt.text(i, v + 1, f'{v:.1f}%', ha='center')

plt.show()


In [ ]:
def plotar_latencia(requisicoes_por_modo):
    latencias = {'reativo': [], 'proativo': [], 'hibrido': []}
    
    for modo, requisicoes in requisicoes_por_modo.items():
        for request in requisicoes:
            latencia = request.endtime - request.starttime
            latencias[modo].append(latencia)
    
    plt.figure(figsize=(10, 6))
    
    for modo, dados in latencias.items():
        ids = list(range(len(dados)))  # Criar uma sequência de 0 a n para o eixo X
        plt.plot(ids, dados, marker='o', label=modo)
    
    plt.xlabel('Número da Requisição')
    plt.ylabel('Latência (time-slots)')
    plt.title('Comparação de Latência entre Modos')
    plt.legend()
    plt.grid(True)
    plt.show()

plotar_latencia(received_requests)